# f1_extract_sentences_with_entities
From the file sentences.txt extract all the sentence who contain entities and write them into .txt file

In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import collections
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def write_list_collected(sentences,s):
    """Write list_sentence_with_country.txt file. """
    outF = open(s, "w")
    for line in sentences :
        # write line to output file
        outF.write(line)
        outF.write('\n')
    outF.close()

In [3]:
def write_list_collected_issue(list_with_two_entities, number):
    """Write list_sentence_with_country.txt file. """
    
    s = "s2e"+str(number)
    outF = open(s+".txt", "w")
    for line in list_with_two_entities :
        # write line to output file
        outF.write(line)
        outF.write('\n')
    outF.close()

In [4]:
def extract_from_txt_sentences(txt_file):
    """ Open file that contains all the sentences."""
    list_tp = open(txt_file)
    return list_tp

In [5]:
def clean_tp(sentence):
    """ Clean the sentence by removing special char."""
    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

In [6]:
def tokenize_sentence(sentence):
    """SPlit the sentence in a way that the entities are together and will be able to be detected."""
    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    list_entities = [s.replace('class="textstory"','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]

    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')

    line = sentence.replace(",","")
    line_splited = word_tokenize(line)
    tokens = tokenizer1.tokenize(line_splited) 
    tokens = tokenizer2.tokenize(tokens) 
    tokens = tokenizer3.tokenize(tokens) 
    tokens = [clean_tp(token) for token in tokens]

    return tokens

In [7]:
def extract_s2e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("sentences_s1e.txt"))
    print(len(sentences))
    sentences = list(set(sentences))
    print(len(sentences))
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s2e = []
    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detetc entity
    
        tokens = tokenize_sentence(s)
        done = False
        for e1 in list_entities: 
            for e2 in list_entities:
                if(set([e1,e2]).issubset(set(tokens)) and e1 != e2):
                    sentences_s2e.append(s)
                    done = True
                    break
            if done:
                break
        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    s = "sentences_s2e.txt"
    write_list_collected(sentences_s2e,s)
    return sentences_s2e

In [8]:
def extract_s1e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("sentences.txt"))
    sentences = list(set(sentences))
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s1e = []
    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detect entity
        tokens = tokenize_sentence(s)
        done = False
        for e1 in list_entities: 
            if(set([e1]).issubset(set(tokens)) ):
                sentences_s1e.append(s)
                done = True
                break

        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    s = "sentences_s1e.txt"
    write_list_collected(sentences_s1e,s)
    return sentences_s1e

In [9]:
def extract_sentences():
    sentences_s1e = extract_s1e_issue()
    sentences_s2e = extract_s2e_issue()
    return sentences_s1e = extract_s1e_issue() , sentences_s2e = extract_s2e_issue()

SyntaxError: invalid syntax (<ipython-input-9-aef259332f78>, line 4)

### Functions to keep only tags needed

In [10]:
def keep_ent_vb_conj(sentence, pattern_verb, pattern_conj, list_entities):
    """ Keep only the patterns needed to find the interactions. """
    pos_tagged = sentence
    token_s = []
    for i in range(len(pos_tagged)):
        w = pos_tagged[i]

        if(set([w[0]]).issubset(set(list_entities))):
            token_s.append(w)
        if(set([w[1]]).issubset(set(pattern_verb))):
            token_s.append(w)
        if(set([w[0]]).issubset(set(pattern_conj))):
            token_s.append(w)

    return token_s

In [11]:
def keep_all_expected_conj(sentence, pattern_verb, pattern_conj, list_entities): 
    tokens = []
    for x in sentence:
        if(x[1]!='IN'):
            tokens.append(x)
    return tokens

In [9]:
def find_pos_tagged_s2e(list_s2e):
    """ Use NLTK to pos_tag all the sentences from list_s2e and return a list of all the sentences pos_tagged"""
    verbs = []
    nnp = []
    pos_tagged = []
    for s in list_s2e:
        s = s.replace('\\t','')
        tokens = tokenize_sentence(s)
        pos_tagged.append(nltk.pos_tag(tokens))  
   
    return pos_tagged

In [21]:
def find_patterns(pos_tagged, function):

    # List of patterns needed
    pattern_verbs = ['VBD','VBN']
    pattern_conj = ['and','with','while']
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    
    # Find all the pattern needed in sentences pos_tagged
    groups = []
    for x in pos_tagged:
        groups.append(function(x, pattern_verbs, pattern_conj, list_entities))

    return groups

In [51]:
def find_verbs_between(pos_tagged):
# Look for all the vebrs between entities : interactions
    verbs_interactions = []
    pattern_verbs = ['VBD','VBN']
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    link = []

    for g in groups:
        for i in range(len(g)-2):
            x = g[i]
            y = g[i+1]
            z = g[i+2]
            if(x[0] in list_entities and y[1] in pattern_verbs and z[0] in list_entities ):
                verbs_interactions.append(y[0])



    s = "verbs_interactions"
    write_list_collected(list(set(verbs_interactions)))

    return list(set(verbs_interactions))

In [52]:
def find_verbs_e_v_e(pos_tagged):
# Look for all the vebrs between entities : interactions
    verbs_interactions = []
    pattern_verbs = ['VBD','VBN']
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    link = []

    for g in groups:
        for i in range(len(g)-2):
            x = g[i]
            y = g[i+1]
            z = g[i+2]
            if(x[0] in list_entities and y[1] in pattern_verbs and z[0] in list_entities ):
                verbs_interactions.append(y[0])
                link.append((str(x[0])+" - "+str(y[0])+" - "+str(z[0])))
    
    s = "link_pattern_e_v_e"
    write_list_collected(link,s)

    s = "verbs_e_v_e"
    write_list_collected(list(set(verbs_interactions)),s)

In [248]:
def find_linked_verbs(sentence, pattern_verbs, pattern_conj, list_entities):
    tokens = []
    for i in range(len(sentence)):
        if(sentence[i][1] != 'IN'):
            tokens.append(sentence[i])
    return tokens


### Find verbs for interaction

In [53]:
find_verbs_e_v_e(pos_tagged)


In [45]:
print(len(pos_tagged))
groups = find_patterns(pos_tagged, keep_all_expected_conj)
print(groups[0])

5198
[('SOUTH AFRICA', 'NNP'), ('and', 'CC'), ('JAPAN', 'NNP'), ('underscored', 'JJ'), ('comparability', 'NN'), ('and', 'CC'), ('CHINA', 'NNP'), ('noted', 'VBD'), ('it', 'PRP'), ('includes', 'VBZ'), ('inter', 'JJ'), ('alia', 'NNS'), ('comparable', 'JJ'), ('efforts', 'NNS'), ('and', 'CC'), ('legally', 'RB'), ('binding', 'VBG'), ('commitments', 'NNS'), ('.', '.')]


In [92]:
def combine_conj(groups, conj_sup, list_entities):
    """ Detect collaboration between two entities linked with and. """
    group = []
    for i in range(len(groups[0])-2):
        x = groups[0][i]
        y = groups[0][i+1]
        z = groups[0][i+2]
        print(x,y,z)

        if(x[0] in list_entities and y[1] in conj_sup and z[0] in list_entities):
            group.append((x[0],z[0]))

    for i in range(len(group)-1):
        if(set(group[i]).intersection(set(group[i+1])) != set()):
            group[i] = list(set(group[i]).union(set(group[i+1])))
            group[i+1] = []

    return group

In [44]:
pos_tagged = find_pos_tagged_s2e(open("sentences_s2e.txt"))

In [112]:
def combine_neighbors(groups):
     """ Detect collaboration between two entities linked ,. """
    group = []
    for i in range(len(groups[0])-1):
        x = groups[0][i]
        y = groups[0][i+1]

        if(x[0] in list_entities and y[0] in list_entities):
            group.append((x[0],y[0]))

    for i in range(len(group)-1):

        if(set(group[i]).intersection(set(group[i+1])) != set()):
            group[i] = list(set(group[i]).union(set(group[i+1])))
            group[i+1] = []

    return group

In [138]:
def combine_by_verbs(groups, verbs_for):
     """ Detect collaboration between two entities linked with verb of support. """
    group = []
    for i in range(len(groups[0])-2):
        x = groups[0][i]
        y = groups[0][i+1]
        z = groups[0][i+2]
        print(x,y,z)
        if(x[0] in list_entities and y[0] in verbs_for and z[0] in list_entities):
            group.append([x[0],z[0]])
            
    for i in range(len(group)-1):
        if(set(group[i]).intersection(set(group[i+1])) != set()):

            group[i] = list(set(group[i]).union(set(group[i+1])))
            group[i+1] = []


    return group

In [249]:
def find_patterns(pos_tagged,func):  
    """ Find all the pattern needed in sentences pos_tagged. Need to call it with find_linked_verbs or find_groups.  """
    # List of patterns needed
    pattern_verbs = ['VBD','VBN']
    pattern_conj = ['and','with','while']
    list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
    print(len(pos_tagged))
    groups = []
    for sentence in pos_tagged:
        groups.append(func(sentence, pattern_verbs, pattern_conj, list_entities))

    return groups

### Functions to find parties with same interests

In [140]:
pos_tagged_s = find_pos_tagged_s2e(s)

In [139]:
s = ["The G-77/CHINA supported by CLIMATE ACTION NETWORK and others, opposed by JAPAN AND CHILE"]
pattern_verbs = ['VBD','VBN']
pattern_conj = ['CC']
list_entities = [s.replace('\n','') for s in list(open('entities_clean.txt'))]
verbs_against = ['opposed']
verbs_for = ['supported']